# Best practices on generating a model
1. Field types and constraints
   1. Use appropriate field types
   2. Add validators when needed
   3. Set reasonable constraints (max_length, etc.)
2. Choices
   1. Use `TextChoices` class (i.e. status fields)
   2. Provides safety and better code organization
3. Meta Class
   1. Define an ordering 
   2. Add useful indexes
   3. Set proper verbose names 
4. Relationships 
   1. Use meaningful `related_name`
   2. Choose the appropriate `on_delete` behavior
5. Internationalization
   1. Use `gettext_lazy` for translatable strings
   2. Add verbose names for all fields 
6. Timestamps
   1. Include `created_at` and `updated_at` fields 
   2. Use `auto_now_add` and `auto_now` 
7. String representation
   1. Implement `__str__` method for readable object representation

In [ ]:
from django.db import models
from django.contrib.auth.models import User
from django.core.validators import MinValueValidator
from django.utils.translation import gettext_lazy as _

class MaterialRequests(models.Model):
    """M101 Requests Model to bring support to M16P"""
    
    class Status(models.TextChoices):
        PENDING = 'PD', _('Pending')
        APPROVED = 'AP', _('Approved')
        REJECTED = 'RJ', _('Rejected')
        COMPLETED = 'CP', _('Completed')

    requesting_user = models.ForeignKey(
        User,
        on_delete=models.CASCADE,
        related_name='material_requests',
        verbose_name=_('Requesting User')
    )
    
    status = models.CharField(
        max_length=2,
        choices=Status.choices,
        default=Status.PENDING,
        verbose_name=_('Status')
    )
    
    request_date = models.DateTimeField(
        auto_now_add=True,
        verbose_name=_('Request Date')
    )
    
    message = models.TextField(
        blank=True,
        verbose_name=_('Message')
    )
    
    part_number = models.CharField(
        max_length=50,
        verbose_name=_('Part Number')
    )
    
    quantity = models.PositiveIntegerField(
        validators=[MinValueValidator(1)],
        verbose_name=_('Quantity')
    )
    
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    
    class Meta:
        verbose_name = _('Material Request')
        verbose_name_plural = _('Material Requests')
        ordering = ['-request_date']
        indexes = [
            models.Index(fields=['request_date']),
            models.Index(fields=['status']),
        ]

    def __str__(self):
        return f"Request {self.id} - {self.part_number} ({self.status})"